<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1


    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance


    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-15 12:42:07
-------------------
qualified stocks: 86
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  1.09 L
Current:  1.39 C
-------------------
Today PnL: 87.62 K (0.63%)
Current PnL: -22.34 L (-14.88%)
CY Booked + Current PnL: -10.43 L (-6.95%)
-------------------
Total profit:  1.54 L
Total loss:  -23.88 L
-------------------
Total Booked + Current PnL: 16.55 L (13.44%)
Total Booked PnL: 38.88 L (31.58%)
Curr Year Booked PnL: 11.90 L (8.55%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 89.52 L (64.34%)
Deployed:  1.23 C
Current:  1.39 C
CAGR/XIRR %: 7.28%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TTKPRESTIG,770.00,98.72,55.0,M-SC,1.50,85595.0,-15182.0,15279.0,1.24,-15.06,17.85,0.09,245.0,-0.99,0.62,12.45,OX40N,NTT,DURABLES
18,CIPLA,1662.74,-20.47,53.0,X-LC,3.70,212090.0,7590.0,15716.0,-0.27,3.71,7.41,11.39,10.0,0.48,1.54,13.00,X40N,ATH,PHARMA
33,ICICIGI,2260.25,-13.38,67.0,X-MC,1.73,148814.0,12681.0,18438.0,8.42,9.31,12.39,22.86,91.0,0.69,1.08,23.10,X40,ATH,INSURANCE
83,VOLTAS,1530.00,0.04,55.0,X-MC,1.06,209925.0,18183.0,19565.0,0.56,9.48,9.32,19.69,99.0,0.93,1.52,16.93,XY25,NTT,AC
56,RELIANCE,1533.00,-14.04,48.0,X-LC,2.97,214968.0,4602.0,24184.0,0.15,2.19,11.25,13.68,37.0,0.19,1.56,19.07,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,ICICIGI,2260.25,-13.38,67.0,X-MC,1.73,148814.0,12681.0,18438.0,8.42,9.31,12.39,22.86,91.0,0.69,1.08,23.10,X40,ATH,INSURANCE
49,NESTLEIND,1377.00,-7.33,61.0,X-LC,2.38,285059.0,19633.0,37172.0,3.63,7.40,13.04,21.40,11.0,0.53,2.06,14.53,XY25,NTT,FMCG
25,GREENPANEL,537.00,219.12,50.0,M-SC,3.92,142520.0,-40558.0,119532.0,2.69,-22.15,83.87,43.14,230.0,-0.34,1.03,34.06,XY24,NTT,MISC
78,UNIONBANK,163.00,-11.63,57.0,M-LC,9.56,159476.0,18636.0,25851.0,2.38,13.23,16.21,31.59,66.0,0.72,1.16,43.22,XY24,NTT,BANKS
5,ASIANPAINT,3465.66,-17.24,44.0,X-LC,5.36,208754.0,-43014.0,96215.0,2.33,-17.08,46.09,21.13,27.0,-0.45,1.51,12.29,X40,ATH,PAINTS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,ICICIPRULI,790.0,-23.40,38.0,X-MC,2.13,131747.0,-4041.0,46797.0,-2.52,-2.98,35.52,31.48,107.0,-0.09,0.95,8.78,X40,ATH,INSURANCE
15,CAMPUS,393.0,-24.03,54.0,M-SC,2.69,158267.0,-9508.0,65348.0,-2.11,-5.67,41.29,33.28,210.0,-0.15,1.15,22.67,XY24,NTT,FOOTWEAR
46,LAOPALA,464.0,95.46,30.0,H-SC,1.51,89345.0,-41760.0,93008.0,-1.69,-31.85,104.10,39.09,142.0,-0.45,0.65,15.07,AR,NTT,CERAMICS
39,INFY,2275.0,-19.60,47.0,X-LC,7.65,313706.0,818.0,170876.0,-1.15,0.26,54.47,54.87,3.0,0.00,2.27,6.93,X40,BTT,IT
7,ATULAUTO,844.0,3651.35,51.0,M-SC,4.75,168589.0,-28543.0,118366.0,-0.90,-14.48,70.21,45.57,236.0,-0.24,1.22,19.53,XY24,NTT,AUTO


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.0,-186.36,68.0,M-SC,17.34,148788.0,-432.0,144607.0,0.33,-0.29,97.19,96.62,208.0,-0.0,1.08,61.02,XY25,NTT,FINANCE
59,SAIL,228.0,42.74,48.0,M-MC,12.29,224721.0,-241.0,167889.0,1.34,-0.11,74.71,74.53,192.0,-0.0,1.63,32.11,XY24,BTT,STEEL


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,105.37,58.0,H-SC,11.51,130032.0,-12447.0,31962.0,1.22,-8.74,24.58,13.70,163.0,-0.39,0.94,53.47,OX40N,NTT,BANKS
76,TTKPRESTIG,770.0,98.72,55.0,M-SC,1.50,85595.0,-15182.0,15279.0,1.24,-15.06,17.85,0.09,245.0,-0.99,0.62,12.45,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.79,51.0,H-SC,1.86,222282.0,-47385.0,83711.0,0.63,-17.57,37.66,13.47,138.0,-0.57,1.61,13.02,XY24,NTT,PAINTS
17,CERA,9475.0,-23.39,42.0,H-SC,2.10,139806.0,-36097.0,78124.0,-0.20,-20.52,55.88,23.89,149.0,-0.46,1.01,20.74,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-24.72,43.0,H-MC,7.02,101926.0,-29909.0,72133.0,0.55,-22.69,70.77,32.03,98.0,-0.41,0.74,14.17,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RECLTD,446.00,47.87,52.0,M-LC,6.30,207570.0,4400.0,37736.0,1.92,2.17,18.18,20.74,55.0,0.12,1.50,7.83,XY25,NTT,FINANCE
31,HINDZINC,730.22,36.19,68.0,M-LC,8.88,221741.0,16665.0,95903.0,0.81,8.13,43.25,54.89,52.0,0.17,1.61,34.18,X5K,ATH,METALS
78,UNIONBANK,163.00,-11.63,57.0,M-LC,9.56,159476.0,18636.0,25851.0,2.38,13.23,16.21,31.59,66.0,0.72,1.16,43.22,XY24,NTT,BANKS
85,WIPRO,420.00,-11.98,55.0,M-LC,5.77,154882.0,3937.0,105521.0,0.56,2.61,68.13,72.51,53.0,0.04,1.12,8.79,XR,NTT,IT
11,BANKINDIA,190.00,-24.22,64.0,H-MC,12.76,195172.0,15364.0,97040.0,2.09,8.54,49.72,62.52,88.0,0.16,1.41,45.54,XR,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HINDZINC,730.22,36.19,68.0,M-LC,8.88,221741.0,16665.0,95903.0,0.81,8.13,43.25,54.89,52.0,0.17,1.61,34.18,X5K,ATH,METALS
11,BANKINDIA,190.00,-24.22,64.0,H-MC,12.76,195172.0,15364.0,97040.0,2.09,8.54,49.72,62.52,88.0,0.16,1.41,45.54,XR,NTT,BANKS
85,WIPRO,420.00,-11.98,55.0,M-LC,5.77,154882.0,3937.0,105521.0,0.56,2.61,68.13,72.51,53.0,0.04,1.12,8.79,XR,NTT,IT
36,INDIAMART,4810.62,-55.39,39.0,H-SC,8.29,124603.0,1267.0,130360.0,0.69,1.03,104.62,106.72,119.0,0.01,0.90,24.65,AR,ATH,MISC
84,WHIRLPOOL,2270.00,-47.61,38.0,M-SC,3.97,88508.0,-2990.0,81746.0,2.26,-3.27,92.36,86.07,223.0,-0.04,0.64,28.53,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-55.15,12.0,X-LC,3.33,142332.0,-134191.0,244256.0,-0.85,-48.53,171.61,39.81,54.0,-0.55,1.03,0.00,XY24,NTT,AUTO
51,QUESS,986.00,-48.21,31.0,X-SC,38.40,51465.0,-13541.0,164467.0,0.36,-20.83,319.57,232.18,198.0,-0.08,0.37,0.36,XY24,NTT,MISC
28,HAVELLS,2069.16,-8.02,34.0,X-MC,1.51,227978.0,-19753.0,94816.0,0.85,-7.97,41.59,30.30,92.0,-0.21,1.65,4.26,X40,ATH,ELECTRICAL
55,RELAXO,1176.00,-41.43,34.0,X-SC,3.76,79787.0,-65373.0,141303.0,-0.61,-45.04,177.10,52.31,136.0,-0.46,0.58,6.80,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-37.44,35.0,X-SC,4.41,93624.0,-35046.0,80348.0,0.34,-27.24,85.82,35.20,219.0,-0.44,0.68,7.46,X40,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VOLTAS,1530.00,0.04,55.0,X-MC,1.06,209925.0,18183.0,19565.0,0.56,9.48,9.32,19.69,99.0,0.93,1.52,16.93,XY25,NTT,AC
64,SIEMENS,4671.50,-3.25,43.0,H-LC,1.28,155365.0,-30730.0,78211.0,0.97,-16.51,50.34,25.51,15.0,-0.39,1.13,14.49,AR,ATH,ELECTRICAL
28,HAVELLS,2069.16,-8.02,34.0,X-MC,1.51,227978.0,-19753.0,94816.0,0.85,-7.97,41.59,30.30,92.0,-0.21,1.65,4.26,X40,ATH,ELECTRICAL
33,ICICIGI,2260.25,-13.38,67.0,X-MC,1.73,148814.0,12681.0,18438.0,8.42,9.31,12.39,22.86,91.0,0.69,1.08,23.10,X40,ATH,INSURANCE
49,NESTLEIND,1377.00,-7.33,61.0,X-LC,2.38,285059.0,19633.0,37172.0,3.63,7.40,13.04,21.40,11.0,0.53,2.06,14.53,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-55.15,12.0,X-LC,3.33,142332.0,-134191.0,244256.0,-0.85,-48.53,171.61,39.81,54.0,-0.55,1.03,0.00,XY24,NTT,AUTO
51,QUESS,986.00,-48.21,31.0,X-SC,38.40,51465.0,-13541.0,164467.0,0.36,-20.83,319.57,232.18,198.0,-0.08,0.37,0.36,XY24,NTT,MISC
82,VBL,671.64,-21.55,38.0,X-LC,10.69,282506.0,-33336.0,145999.0,0.27,-10.55,51.68,35.67,5.0,-0.23,2.05,1.84,X40N,ATH,FMCG
9,BAJAJHFL,181.50,-15.75,36.0,X-MC,5.66,177399.0,-23107.0,120259.0,0.88,-11.52,67.79,48.45,90.0,-0.19,1.28,2.30,X40N,ATH,FINANCE
41,ITC,452.00,-39.61,44.0,X-LC,2.83,195527.0,-4611.0,25497.0,0.77,-2.30,13.04,10.44,4.0,-0.18,1.42,3.13,X40,NTT,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-48.21,31.0,X-SC,38.40,51465.0,-13541.0,164467.0,0.36,-20.83,319.57,232.18,198.0,-0.08,0.37,0.36,XY24,NTT,MISC
55,RELAXO,1176.00,-41.43,34.0,X-SC,3.76,79787.0,-65373.0,141303.0,-0.61,-45.04,177.10,52.31,136.0,-0.46,0.58,6.80,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-37.44,35.0,X-SC,4.41,93624.0,-35046.0,80348.0,0.34,-27.24,85.82,35.20,219.0,-0.44,0.68,7.46,X40,NTT,FOOTWEAR
16,CAMS,5211.76,-5.16,50.0,X-SC,1.16,107870.0,5866.0,38057.0,1.92,5.75,35.28,43.06,122.0,0.15,0.78,23.85,X40N,ATH,MISC
32,HONAUT,58357.33,-23.25,44.0,X-SC,2.32,108120.0,-19818.0,66948.0,-0.14,-15.49,61.92,36.84,143.0,-0.30,0.78,10.69,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-29.97,45.0,X-LC,12.32,279293.0,-66663.0,133362.0,0.74,-19.27,47.75,19.28,1.0,-0.50,2.02,3.25,X40,ATH,IT
39,INFY,2275.00,-19.60,47.0,X-LC,7.65,313706.0,818.0,170876.0,-1.15,0.26,54.47,54.87,3.0,0.00,2.27,6.93,X40,BTT,IT
41,ITC,452.00,-39.61,44.0,X-LC,2.83,195527.0,-4611.0,25497.0,0.77,-2.30,13.04,10.44,4.0,-0.18,1.42,3.13,X40,NTT,FMCG
82,VBL,671.64,-21.55,38.0,X-LC,10.69,282506.0,-33336.0,145999.0,0.27,-10.55,51.68,35.67,5.0,-0.23,2.05,1.84,X40N,ATH,FMCG
1,ABB,7934.00,-40.16,50.0,H-LC,7.41,250032.0,-11587.0,130792.0,1.75,-4.43,52.31,45.57,7.0,-0.09,1.81,6.39,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
20,COFFEEDAY,80.00,-45.67,48.0,L-SC,28.66,80544.0,-33005.0,73053.0,-0.71,-29.07,90.70,35.27,268.0,-0.45,0.58,97.13,XR,NTT,HOTELS
48,MASFIN,398.61,-21.41,39.0,H-SC,9.86,89460.0,-8520.0,30121.0,-0.47,-8.70,33.67,22.05,152.0,-0.28,0.65,30.04,XR,ATH,FINANCE
77,UJJIVANSFB,60.00,105.37,58.0,H-SC,11.51,130032.0,-12447.0,31962.0,1.22,-8.74,24.58,13.70,163.0,-0.39,0.94,53.47,OX40N,NTT,BANKS
68,SURYODAY,240.00,57.89,53.0,H-SC,9.92,144544.0,-34527.0,100256.0,-0.60,-19.28,69.36,36.71,135.0,-0.34,1.05,43.42,XR,NTT,BANKS
33,ICICIGI,2260.25,-13.38,67.0,X-MC,1.73,148814.0,12681.0,18438.0,8.42,9.31,12.39,22.86,91.0,0.69,1.08,23.10,X40,ATH,INSURANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-186.36,68.0,M-SC,17.34,148788.0,-432.0,144607.0,0.33,-0.29,97.19,96.62,208.0,-0.00,1.08,61.02,XY25,NTT,FINANCE
68,SURYODAY,240.00,57.89,53.0,H-SC,9.92,144544.0,-34527.0,100256.0,-0.60,-19.28,69.36,36.71,135.0,-0.34,1.05,43.42,XR,NTT,BANKS
31,HINDZINC,730.22,36.19,68.0,M-LC,8.88,221741.0,16665.0,95903.0,0.81,8.13,43.25,54.89,52.0,0.17,1.61,34.18,X5K,ATH,METALS
4,ANGELONE,3033.00,12.24,65.0,X-SC,9.92,199071.0,8065.0,49628.0,0.96,4.22,24.93,30.21,157.0,0.16,1.44,25.99,X40N,NTT,FINANCE
11,BANKINDIA,190.00,-24.22,64.0,H-MC,12.76,195172.0,15364.0,97040.0,2.09,8.54,49.72,62.52,88.0,0.16,1.41,45.54,XR,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.50
1,25,44.02
2,50,75.92


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.35
LC    33.74
MC    21.92
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.67
X40      14.82
XY25     12.18
XR       10.99
X40N     10.10
AR        8.22
OX40N     7.68
X200      1.81
X5K       1.61
SR        1.11
MH        0.82
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.57
X-LC    22.57
X-MC    16.14
M-SC    12.71
M-LC     5.39
H-LC     4.75
X-SC     4.63
H-MC     3.84
M-MC     1.63
L-SC     1.44
L-LC     1.03
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.10,-5.46,40.22
IT,12.98,-17.20,80.60
FINANCE,12.21,-12.12,62.94
MISC,6.93,-20.02,86.51
BANKS,6.46,-11.90,70.77
PAINTS,5.61,-19.19,36.96
ELECTRICAL,5.37,-11.04,50.00
INSURANCE,3.88,-3.27,41.88
AUTO,2.82,-47.55,118.08


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3341269.0,22
XR,1394857.0,14
AR,1194667.0,9
X40,791480.0,10
X40N,633096.0,8
OX40N,587390.0,10
XY25,499697.0,8
SR,267575.0,2
X5K,95903.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3471546.0,25
M-SC,1288887.0,15
X-LC,1122331.0,13
X-MC,1097623.0,11
X-SC,540751.0,6
H-MC,350100.0,3
H-LC,284279.0,3
M-LC,265011.0,4
L-SC,262085.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1207894.0      6
           XR         828635.0      7
           AR         821255.0      5
M-SC       XY24       698170.0      6
X-MC       XY24       587311.0      4
X-LC       X40        494859.0      5
           XY24       308760.0      2
X-SC       X40N       295936.0      4
H-SC       OX40N      275075.0      4
           SR         267575.0      2
X-LC       X40N       216901.0      3
X-MC       X40        216273.0      4
H-LC       AR         209003.0      2
H-MC       XY24       180927.0      1
X-MC       XY25       173780.0      2
L-SC       XR         172112.0      2
M-MC       XY24       167889.0      1
X-SC       XY24       164467.0      1
M-SC       AR         164409.0      2
           OX40N      150209.0      4
           XY25       144607.0      1
           XR         131492.0      2
X-MC       X40N       120259.0      1
M-LC       XR         105521.0      1
X-LC       XY25       101811.0      3
H-MC       XR          97040.0      1
M-LC       X5K         95903.0      1
L-SC       OX40N       89973.0      1
X-SC       X40         80348.0      1
H-LC       X200        75276.0      1
H-MC       OX40N       72133.0      1
H-SC       MH          71112.0      1
L-MC       XR          60057.0      1
L-LC       XY25        41763.0      1
M-LC       XY25        37736.0      1
           XY24        25851.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 32.0 seconds
